In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
FILE_PATH = os.path.join(os.path.abspath('.'))

In [ ]:
df_train = pd.read_csv(os.path.join(FILE_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(FILE_PATH, 'test.csv'))

In [ ]:
df_train['IN_TREINEIRO'].value_counts()
## Desbalanceamento
# Classe 0 (Não treineiro) 11947 : 87%
# Classe 1 (Treineiro) 1783 : 13%

In [ ]:
#Possíveis features para a classificação

df_train.corr()['IN_TREINEIRO'] > 0.20 
## TP_ST_CONCLUSAO : Alta correlação posistiva
df_train.corr()['IN_TREINEIRO'] < -0.20 
## NU_IDADE / TP_ANO_CONCLUIU / TP_ESCOLA : Alta correlação negativa

In [ ]:
# Tratamento do desbalanceamento do target
from imblearn.over_sampling import SMOTE
df_train_copy = df_train.copy()
df_test_copy = df_test.copy()

target = df_train_copy['IN_TREINEIRO']
features = df_train_copy[['NU_IDADE', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ST_CONCLUSAO']]

# Estratégia adotada de incrementar novos dados a classe minoritaria
smote = SMOTE(sampling_strategy="minority")

X_smote, y_smote = smote.fit_resample(features, target)

In [ ]:
# Verificação do balançeamento
y_smote.value_counts()

In [ ]:
## Padronização das variáveis
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()  
X_train = sc.fit_transform(X_smote)  
X_test = sc.transform(df_test_copy[['NU_IDADE', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ST_CONCLUSAO']])

In [ ]:
## Busca pelos melhores parametros
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

estimator = RandomForestRegressor()
param_grid = { 
        "n_estimators" : [100, 200, 300, 400, 500],
        "max_depth" : [2, 4, 6, 8, 10],
        "max_features" : ["auto", "sqrt", "log2"],
        "min_samples_split" : [20, 40, 60, 80, 100],
        "bootstrap": [True, False],
        }

grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

grid.fit(X_train, y_train)

grid.best_score_ , grid.best_params_

In [ ]:
## Utilização dos melhores parametros encontrados
from sklearn.ensemble import RandomForestRegressor
estimator = RandomForestRegressor(
                                max_depth=8,
                                n_estimators=500,
                                 )
estimator.fit(X_train, y_smote)
estimator.predict(X_test)

In [ ]:
answer = pd.DataFrame(data={'NU_INSCRICAO':df_test['NU_INSCRICAO'],
                  'IN_TREINEIRO':estimator.predict(X_test)})
answer.to_csv(os.path.join(FILE_PATH, 'answer.csv'), sep=',', index=False)